In [2]:
import os
import pickle
import argparse
import skipthoughts

In [ ]:
checkpoint_path = 'checkpoints/netG__epoch_25.pth'

In [3]:
model = skipthoughts.load_model()

Loading model parameters...
Compiling encoders...
Loading tables...
Packing up...


NameError: name 'caption_file' is not defined

In [ ]:
netG = NetG()
netG.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))

### train sample

### val sample

### test sample

In [18]:
import pickle

In [22]:
pickle.loads(open('./data/datasets/products/y_vocab.pickle', 'rb').read())

{'-1': 8,
 'All Beauty': 26,
 'All Electronics': 21,
 'Amazon Fashion': 32,
 'Appliances': 38,
 'Arts Crafts & Sewing': 19,
 'Automotive': 18,
 'Baby': 16,
 'Baby Products': 31,
 'Beauty': 17,
 'Books': 0,
 'Buy a Kindle': 45,
 'CDs & Vinyl': 5,
 'Camera & Photo': 27,
 'Car Electronics': 36,
 'Cell Phones & Accessories': 11,
 'Clothing Shoes & Jewelry': 2,
 'Collectible Coins': 46,
 'Collectibles & Fine Art': 34,
 'Computers': 20,
 'Digital Music': 33,
 'Electronics': 14,
 'Entertainment': 47,
 'Furniture & D&#233;cor': 43,
 'GPS & Navigation': 40,
 'Gift Cards': 37,
 'Grocery & Gourmet Food': 23,
 'Health & Personal Care': 10,
 'Home & Kitchen': 9,
 'Home Improvement': 30,
 'Industrial & Scientific': 25,
 'Kindle Store': 44,
 'Kitchen & Dining': 24,
 'Luxury Beauty': 41,
 'MP3 Players & Accessories': 35,
 'Magazine Subscriptions': 42,
 'Movies & TV': 1,
 'Musical Instruments': 6,
 'Office & School Supplies': 15,
 'Office Products': 12,
 'Patio Lawn & Garden': 28,
 'Pet Supplies': 22,


In [24]:
import gzip
g = gzip.open('./data/datasets/products/metadata.json.gz', 'r')

In [29]:
with gzip.open('./data/datasets/products/metadata.json.gz', 'r') as f:
    for i, l in enumerate(f):
        pass

In [30]:
i

9430087